### Project examining All-Time Olympic Medal counts
#### by Erika Harrell

In [67]:
# importing Python libraries
import requests 
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
#show all output in each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [68]:
#scraping table from Wikipedia site
url = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"
#sending url to requests to create HTML object
response = requests.get(url)
#using Beautiful Soup's HTML parser to parse content in HTML object
soup = BeautifulSoup(response.content, "html.parser")
#finding all objects in URL with table label
tables = soup.find_all("table")
# getting the second element of tables wihch is the first table on the wikipdeia page
df1 = pd.read_html(StringIO(str(tables[1])))
# printing element
print(df1)

[                                            Team Summer Olympic Games  \
                                 Team (IOC code)                  No.   
0                              Afghanistan (AFG)                   16   
1                                  Albania (ALB)                   10   
2                                  Algeria (ALG)                   15   
3                                Argentina (ARG)                   26   
4                                  Armenia (ARM)                    8   
..                                           ...                  ...   
158       Individual Neutral Athletes (AIN)[AIN]                    1   
159      Independent Olympic Athletes (IOA)[IOA]                    3   
160  Independent Olympic Participants (IOP)[IOP]                    1   
161                        Mixed team (ZZX)[ZZX]                    3   
162                                       Totals                   30   

                                                 

In [69]:
#get information about df1
type(df1)
len(df1)
type(df1[0])

list

1

pandas.core.frame.DataFrame

In [70]:
#get dataframe out of list
df = df1[0]
type(df)

pandas.core.frame.DataFrame

In [71]:
# get information about dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 16 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   (Team, Team (IOC code))                      163 non-null    object
 1   (Summer Olympic Games, No.)                  163 non-null    int64 
 2   (Summer Olympic Games, Unnamed: 2_level_1)   163 non-null    int64 
 3   (Summer Olympic Games, Unnamed: 3_level_1)   163 non-null    int64 
 4   (Summer Olympic Games, Unnamed: 4_level_1)   163 non-null    int64 
 5   (Summer Olympic Games, Unnamed: 5_level_1)   163 non-null    int64 
 6   (Winter Olympic Games, No.)                  163 non-null    int64 
 7   (Winter Olympic Games, Unnamed: 7_level_1)   163 non-null    int64 
 8   (Winter Olympic Games, Unnamed: 8_level_1)   163 non-null    int64 
 9   (Winter Olympic Games, Unnamed: 9_level_1)   163 non-null    int64 
 10  (Winter Olympi

In [72]:
# dropping the last row of the dataframe
df =df.iloc[:-1]
print(df)

                                            Team Summer Olympic Games  \
                                 Team (IOC code)                  No.   
0                              Afghanistan (AFG)                   16   
1                                  Albania (ALB)                   10   
2                                  Algeria (ALG)                   15   
3                                Argentina (ARG)                   26   
4                                  Armenia (ARM)                    8   
..                                           ...                  ...   
157                          Zimbabwe (ZIM)[ZIM]                   15   
158       Individual Neutral Athletes (AIN)[AIN]                    1   
159      Independent Olympic Athletes (IOA)[IOA]                    3   
160  Independent Olympic Participants (IOP)[IOP]                    1   
161                        Mixed team (ZZX)[ZZX]                    3   

                                                  

In [73]:
# get column names
df.columns

MultiIndex([(                'Team',     'Team (IOC code)'),
            ('Summer Olympic Games',                 'No.'),
            ('Summer Olympic Games',  'Unnamed: 2_level_1'),
            ('Summer Olympic Games',  'Unnamed: 3_level_1'),
            ('Summer Olympic Games',  'Unnamed: 4_level_1'),
            ('Summer Olympic Games',  'Unnamed: 5_level_1'),
            ('Winter Olympic Games',                 'No.'),
            ('Winter Olympic Games',  'Unnamed: 7_level_1'),
            ('Winter Olympic Games',  'Unnamed: 8_level_1'),
            ('Winter Olympic Games',  'Unnamed: 9_level_1'),
            ('Winter Olympic Games', 'Unnamed: 10_level_1'),
            (      'Combined total',                 'No.'),
            (      'Combined total', 'Unnamed: 12_level_1'),
            (      'Combined total', 'Unnamed: 13_level_1'),
            (      'Combined total', 'Unnamed: 14_level_1'),
            (      'Combined total', 'Unnamed: 15_level_1')],
           )

In [74]:
#examining columns
df.head(1)
df.columns[0]
#getting type of upper level columns
print(type(df['Team']))
print(type(df['Summer Olympic Games']))
print(type(df['Winter Olympic Games']))
print(type(df['Combined total']))


Team Summer Olympic Games                     \
     Team (IOC code)                  No. Unnamed: 2_level_1   
0  Afghanistan (AFG)                   16                  0   

                                                            \
  Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1   
0                  0                  2                  2   

  Winter Olympic Games                                        \
                   No. Unnamed: 7_level_1 Unnamed: 8_level_1   
0                    0                  0                  0   

                                         Combined total                      \
  Unnamed: 9_level_1 Unnamed: 10_level_1            No. Unnamed: 12_level_1   
0                  0                   0             16                   0   

                                                               
  Unnamed: 13_level_1 Unnamed: 14_level_1 Unnamed: 15_level_1  
0                   0                   2                   2

('Team', 'Team (IOC code)')

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [75]:
#creating new dataframe from nested dataframes in df

